# Setup
Set up your API keys (e.g., OpenAI API key) in a .env file or as environment variables.

In [ ]:
import os
from dotenv import load_dotenv
from env_utils import doublecheck_env, doublecheck_pkgs

load_dotenv()

# os.environ["GROQ_API_KEY"] = "your_api_key_here"
# Check and print results
doublecheck_env(".env")  # check environmental variables
doublecheck_pkgs(pyproject_path="pyproject.toml", verbose=True)   # check packages


# 1. Simple LLM Chain: Customer Inquiry Routing
A common banking scenario is routing customer inquiries to the appropriate department (e.g., loans, credit cards, technical support). We'll use LCEL to build a simple classification chain.

**Concept**: Chaining a prompt template and an LLM using the pipe | operator.

In [ ]:
#from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the model
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

# Define the banking-specific prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service assistant at a bank. Classify the following customer inquiry into one of these categories: 'Loan Department', 'Credit Card Services', 'Online Banking Support', 'General Inquiry'. Only return the category name."),
        ("human", "{user_inquiry}"),
    ]
)

# Define the output parser
output_parser = StrOutputParser()

# Build the LCEL chain
# The output of prompt_template becomes the input to llm, and the output of llm becomes the input to output_parser
classification_chain = prompt_template | llm | output_parser

# Invoke the chain with a banking-related inquiry
inquiry_1 = "I can't log in to my mobile app, it keeps showing an error code."
response_1 = classification_chain.invoke({"user_inquiry": inquiry_1})
print(f"Inquiry: {inquiry_1}")
print(f"Category: {response_1}")

inquiry_2 = "What are the interest rates for a home loan?"
response_2 = classification_chain.invoke({"user_inquiry": inquiry_2})
print(f"\nInquiry: {inquiry_2}")
print(f"Category: {response_2}")


# 2. Using `RunnableParallel` for Compliance Checks
Banks often need to run multiple checks on a single piece of input, such as a customer's request. We can process these checks in parallel to save time.

**Concept**: Use `RunnableParallel` to send an input to multiple independent chains concurrently.

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Define specialized prompts for different banking checks
fraud_check_prompt = ChatPromptTemplate.from_template(
    "Analyze the transaction request: '{transaction_details}'. Is this likely fraudulent? Respond with 'Yes' or 'No' and a brief reason."
)

compliance_check_prompt = ChatPromptTemplate.from_template(
    "Analyze the transaction request: '{transaction_details}'. Does it comply with anti-money laundering (AML) regulations? Respond with 'Compliant' or 'Non-Compliant' and a brief reason."
)

# Create the parallel chains
# RunnablePassthrough allows the original input (transaction_details) to be passed to both branches
parallel_checks = RunnableParallel(
    fraud_status=fraud_check_prompt | llm | output_parser,
    aml_status=compliance_check_prompt | llm | output_parser
)

# Invoke the parallel chain
transaction_details = "Transfer of $50,000 to an unverified international account with no prior transaction history."
results = parallel_checks.invoke({"transaction_details": transaction_details})
print(f"Transaction Details: {transaction_details}")
print(f"Results: {results}")


# 3. Using RunnableLambda for Data Pre-processing (Credit Scoring)
Before a loan application is processed by an LLM or a model, it might need data cleaning or reformatting. RunnableLambda lets us insert custom Python functions into our LCEL chain.

**Concept**: Wrapping a custom Python function to make it a compatible Runnable component.

In [ ]:
from langchain_core.runnables import RunnableLambda

# A custom Python function to clean and format a credit report summary
def format_credit_data(report_summary: str) -> dict:
    # In a real scenario, this function would handle complex parsing/cleaning
    print(f"--- Processing raw data: {report_summary[:30]}...")
    # Simulate data formatting for the LLM
    formatted_data = {
        "credit_score": report_summary.split('Score: ')[1].split('.')[0],
        "late_payments": "Yes" if "late payments" in report_summary.lower() else "No"
    }
    print(f"--- Formatted data: {formatted_data}")
    return formatted_data

# Define the prompt that takes the formatted dictionary as input
credit_decision_prompt = ChatPromptTemplate.from_template(
    "A potential loan applicant has a credit score of {credit_score} and late payments status is {late_payments}. Provide a brief recommendation for a loan officer (Approve/Deny/Review)."
)

# Build the chain: lambda function -> prompt -> llm -> parser
# The lambda function runs first, and its output (a dictionary) is passed to the prompt
credit_chain = RunnableLambda(format_credit_data) | credit_decision_prompt | llm | output_parser

# Invoke the chain
raw_report = "Customer credit report: Score: 720. No recent late payments. Credit utilization is 30%."
decision = credit_chain.invoke(raw_report)
print(f"\nLoan Officer Recommendation: {decision}")


# 4. Full RAG Chain: Financial Document Q&A
This advanced example demonstrates a complete Retrieval-Augmented Generation (RAG) pipeline for answering questions about a bank's internal policy documents, ensuring accuracy and compliance.

**Concept**: Combining retrieval (data lookup) and generation (LLM response) into a single, seamless LCEL chain.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
#from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
#from pydantic import SecretStr

# 1. Load a dummy banking policy document (replace with real PDF path)
# A dummy text is used here for demonstration.
# In a real case, load an actual PDF file, e.g., 'internal_banking_policy.pdf'
# For now, we'll simulate the process with a simple string
policy_text = """
Internal Banking Policy on Account Freezing (Effective 2025):
Accounts may be frozen in cases of suspected fraud, anti-money laundering (AML) violations, or court orders.
The compliance department is responsible for initiating freezes.
Customers must be notified within 24 hours of a freeze via registered mail and email.
To unfreeze an account, the customer must visit a branch with two forms of valid ID and proof of transaction legitimacy.
"""

# Simulate document creation and splitting
from langchain_core.documents import Document

documents = [Document(page_content=policy_text)]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(documents)

# 2. Embed and store the document chunks (using in-memory Chroma for simplicity)
#embeddings = OpenAIEmbeddings()

embeddings = HuggingFaceBgeEmbeddings(encode_kwargs={'normalize_embeddings': True},model_name="BAAI/bge-m3")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

# 3. Define the RAG prompt template
rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a bank policy expert. Use ONLY the provided internal policy context to answer the customer's question. If the answer is not in the context, state that the information is unavailable."),
        ("human", "Context: {context}\n\nQuestion: {question}"),
    ]
)

# 4. Build the RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | output_parser
)

# Invoke the RAG chain with a banking-related question
question = "What is the procedure for a customer to unfreeze their account?"
rag_response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {rag_response}")

question_unrelated = "What is the bank's stock price today?"
rag_response_unrelated = rag_chain.invoke(question_unrelated)
print(f"\nQuestion: {question_unrelated}")
print(f"Answer: {rag_response_unrelated}")
